## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import requests
from bs4 import BeautifulSoup

import json
import re

import pandas as pd
import numpy as np

import pytz
import glob
from datetime import datetime
from tableauscraper import TableauScraper as TS

## Download

Paths to Dashboard

In [6]:
host_url = "https://public.tableau.com"
path = "/views/COVID_Case_Dashboard_English/Overview"

Retrive Dashboard data

In [7]:
url = f"{host_url}{path}"

In [8]:
ts = TS()
ts.loads(url)

AttributeError: 'NoneType' object has no attribute 'text'

In [9]:
workbook = ts.getWorkbook()

In [10]:
sheet = next(w for w in workbook.worksheets if w.name == "City")

In [11]:
df = sheet.data

In [12]:
city_df = df[df["Measure Names-alias"] == "Cases"][
    [
        "City -value",
        "Measure Values-alias",
    ]
].rename(columns={"City -value": "area", "Measure Values-alias": "confirmed_cases"})

In [13]:
city_df

,area,confirmed_cases
9,Kettleman City,447
10,Other Area,481
11,Stratford,500
12,%null%,819
13,Armona,"1,632"
14,Avenal,"7,329"
15,Corcoran,"11,013"
16,Lemoore,"11,107"
17,Hanford,"23,737"


In [14]:
df2 = next(w for w in workbook.worksheets if w.name == "Totals Cases & Deaths").data

In [15]:
prison_df = (
    df2[
        (df2["Measure Names-alias"] == "Cases")
        & (df2["type-alias"] == "State Correctional Facilities")
    ]
    .rename(columns={"type-alias": "area", "Measure Values-alias": "confirmed_cases"})[
        ["area", "confirmed_cases"]
    ]
    .replace("State Correctional Facilities", "Correctional facilities")
)

In [16]:
prison_df

,area,confirmed_cases
4,Correctional facilities,"9,997"


Reorder columns, Combine and add date, Kings County, and fix text

In [17]:
df_final = pd.concat([city_df, prison_df])

In [18]:
df_final["county"] = "Kings"
df_final = df_final[["county", "area", "confirmed_cases"]]

In [19]:
df_final

,county,area,confirmed_cases
9,Kings,Kettleman City,447
10,Kings,Other Area,481
11,Kings,Stratford,500
12,Kings,%null%,819
13,Kings,Armona,"1,632"
14,Kings,Avenal,"7,329"
15,Kings,Corcoran,"11,013"
16,Kings,Lemoore,"11,107"
17,Kings,Hanford,"23,737"
4,Kings,Correctional facilities,"9,997"


Remove any commas from cases column

In [20]:
df_final["confirmed_cases"] = df_final.confirmed_cases.replace(",", "", regex=True)

## Vet

In [27]:
df_final = df_final[df_final.area != "%null%"]

In [28]:
default_kings_len = 9

In [29]:
try:
    assert not len(df_final) > default_kings_len
except AssertionError:
    raise AssertionError("Kings County has more rows than before")

In [30]:
try:
    assert not len(df_final) < default_kings_len
except AssertionError:
    raise AssertionError("Kings County has missing row(s)")

## Export

Set the date

In [31]:
tz = pytz.timezone("America/Los_Angeles")

In [32]:
today = datetime.now(tz).date()

In [33]:
slug = "kings"

In [34]:
df_final["county_date"] = today

/var/folders/zd/yxs_1mzx1pn4j9kd8cq39n4r0000gn/T/ipykernel_52502/1055557712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["county_date"] = today


In [35]:
df_final.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [36]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [37]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [38]:
df = pd.concat(df_list).sort_values(["date", "area"]).drop_duplicates()

In [39]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)